In [1]:
import copy 
import numpy as np
import math
from geojson import LineString, Feature, FeatureCollection, dump

In [2]:
with open("input.txt") as f:
    instructions = [x.strip('\n') for x in f.readlines()]

In [3]:
def distance(p1, p2):
    """ 
    https://en.wikipedia.org/wiki/Chebyshev_distance
    """
    return max(abs(p2[0] - p1[0]), abs(p2[1] - p1[1]))


# Part 1

In [4]:
head = [[0, 0]] # x, y
tail = [[0, 0]]

for ins in instructions:
    direction, steps = ins.split(' ')
    
    for _ in range(int(steps)):
        c = copy.copy(head[-1])
        match direction:
            case "R":
                c[0] += 1
            case "U":
                c[1] += 1
            case "L":
                c[0] -= 1
            case "D":
                c[1] -= 1
            case _:
                print("Code not found")
        head.append(c)
        
        if distance(head[-1], tail[-1]) >= 2:
            tail.append(copy.copy(head[-2]))

In [5]:
# for changing by index we need lists above, for unqiue coordindates
# convert [x, y] to tuples (x, y), b/c we can't use a set() on lists.
len(set([(x[0], x[1]) for x in tail]))

5960

# Part 2

Works also for part 1.

In [6]:
# prepare N=10 knots
knots = {}
for i in range(0, 10):
    knots[i] = [[0, 0]]

for ins in instructions:
    direction, steps = ins.split(' ')
    
    for _ in range(int(steps)):
        # move head
        c = copy.copy(knots[0][-1])
        match direction:
            case "R":
                c[0] += 1
            case "U":
                c[1] += 1
            case "L":
                c[0] -= 1
            case "D":
                c[1] -= 1
            case _:
                print("Code not found")
        knots[0].append(c)
        
        # move N knots
        for i, knot in knots.items():
            
            # head is moved above
            if i == 0:
                continue
                
            if distance(knot[-1], knots[i-1][-1]) >= 2:
                dists = {}
                t = knots[i-1][-1]
                n = copy.copy(knot[-1])

                # diagonal move, chose the one with smallest distance
                if(knot[-1][0] != knots[i-1][-1][0] and 
                  knot[-1][1] != knots[i-1][-1][1]):        
             
                    # up-right
                    n_ur = [n[0]+1, n[1]+1]
                    ur_d = distance(n_ur, t)
                    dists[ur_d] = n_ur

                    # down-right
                    n_dr = [n[0]+1, n[1]-1]
                    dr_d = distance(n_dr, t)   
                    dists[dr_d] = n_dr

                    # up-left
                    n_ul = [n[0]-1, n[1]+1]
                    ul_d = distance(n_ul, t)
                    dists[ul_d] = n_ul

                    # down-left
                    n_dl = [n[0]-1, n[1]-1]
                    dl_d = distance(n_dl, t)
                    dists[dl_d] = n_dl
                else:
                    # up
                    n_u = [n[0], n[1]+1]
                    u_d = distance(n_u, t)
                    dists[u_d] = n_u

                    # down
                    n_d = [n[0], n[1]-1]
                    d_d = distance(n_d, t)   
                    dists[d_d] = n_d

                    # left
                    n_l = [n[0]-1, n[1]]
                    l_d = distance(n_l, t)
                    dists[l_d] = n_l

                    # right
                    n_r = [n[0]+1, n[1]]
                    r_d = distance(n_r, t)
                    dists[r_d] = n_r

                min_dists = min(dists.keys())
                knot.append(dists[min_dists])

    # Enable for printing, change arr size according to input
    if False:
        arr = np.empty([10, 10], dtype=str)
        arr[:] = '.'

        for i, knot in knots.items():
            p = knot[-1]
            arr[p[0], p[1]] ="H" if (i==0) else str(i)
        arr[0, 0] = 's'
        
        print(f"== {ins} ==")
        arr = np.rot90(arr)
        w, h = np.shape(arr)
        for y in range(h):
            print(''.join(arr[y, :]))
        print("")

In [7]:
print(len(set([(x[0], x[1]) for x in knots[1]])))
print(len(set([(x[0], x[1]) for x in knots[9]])))

5960
2327


In [8]:
def route_to_geojson(fn, points, props={}):

    p2 = []
    for p in points:
        p2.append([p[0]/100, p[1]/100]) # prevent lat >90
    
    tuples = [(x[0], x[1]) for x in p2]
    ls = LineString(tuples)
    features = []
    features.append(Feature(geometry=ls, properties=props))
    feature_collection = FeatureCollection(features)

    with open(f'{fn}.geojson', 'w') as f:
        dump(feature_collection, f)

for i, knot in knots.items():
    route_to_geojson(f'{i}', knot)
        
#route_to_geojson('head', head)
#route_to_geojson('tail', tail, {
#    "stroke": "#ff0000",
#    "stroke-width": 2,
#    "stroke-opacity": 1
#})